In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [4]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > DESIRED_ACCURACY):
      print(f"/nReached {DESIRED_ACCURACY} so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()

--2021-08-06 06:50:59--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 172.217.2.112, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-08-06 06:50:59 (153 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [9]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (2,2), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D((2,2)),
  tf.keras.layers.Conv2D(128, (2,2), activation='relu'),
  tf.keras.layers.MaxPooling2D((2,2)),
  tf.keras.layers.Conv2D(256, (2,2), activation='relu'),
  tf.keras.layers.MaxPooling2D((2,2)),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics='accuracy')

In [14]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(150,150),
    class_mode='binary',
    batch_size=20
)

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [15]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
    train_generator,
    steps_per_epoch=4,
    callbacks=[callbacks],
    epochs=20,
    verbose=1)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
4/4 [==============================] - 8s 2s/step - loss: 8.5287 - accuracy: 0.4750
Epoch 2/20
4/4 [==============================] - 6s 2s/step - loss: 0.5889 - accuracy: 0.6375
Epoch 3/20
4/4 [==============================] - 6s 2s/step - loss: 0.4952 - accuracy: 0.6875
Epoch 4/20
4/4 [==============================] - 6s 2s/step - loss: 0.5484 - accuracy: 0.7500
Epoch 5/20
4/4 [==============================] - 6s 2s/step - loss: 0.1839 - accuracy: 0.9125
Epoch 6/20
4/4 [==============================] - 6s 2s/step - loss: 0.3317 - accuracy: 0.8500
Epoch 7/20
4/4 [==============================] - 6s 2s/step - loss: 0.1598 - accuracy: 0.9625
Epoch 8/20
4/4 [==============================] - 6s 2s/step - loss: 0.1087 - accuracy: 0.9500
Epoch 9/20
4/4 [==============================] - 6s 2s/step - loss: 0.1069 - accuracy: 0.9750
Epoch 10/20
4/4 [==============================] - 6s 2s/step - loss: 0.0652 - accuracy: 0.9750
Epoch 11/20
4/4 [==============================] 